In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataset import Dataset, log_det_fit_fn
from submodular_monotone_net import SubmodularMonotoneNet
from torch.utils.data import random_split, DataLoader
import torch

In [3]:
DATASET_SIZE = int(1e4)
SAMPLE_SIZE = int(1e4)
Z_SIZE = 10

dataset = Dataset(DATASET_SIZE, SAMPLE_SIZE, Z_SIZE, log_det_fit_fn)

In [4]:
dataset[0]

(tensor([False, False,  True,  ..., False, False,  True]), tensor(13.6579))

In [5]:
train_size = int(0.8 * len(dataset))  # 80% for training
test_size = len(dataset) - train_size  # Remaining for testing

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoader for training and testing
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
from submodular_monotone_net import SubmodularMonotoneNet
monotone_network = SubmodularMonotoneNet(10, 0.5, SAMPLE_SIZE, Z_SIZE, [1, 100, 100, 1], dataset)

/Users/devarora/ml/submodular_nn/increasing_concave_net.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(
/Users/devarora/ml/submodular_nn/increasing_concave_net.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(


In [7]:
monotone_network.input_size

10000

In [8]:
opt = torch.optim.Adam(
    params=monotone_network.parameters(), lr=1e-3
)

num_epochs = 100
criterion = torch.nn.MSELoss()

for epoch in range(num_epochs):
    monotone_network.train()  # Set the model to training mode
    running_loss = 0.0
    for inputs, labels in train_loader:
        opt.zero_grad()  # Zero the parameter gradients
        
        outputs = monotone_network(inputs.to(torch.float32))  # Forward pass
        
        loss = criterion(outputs, labels)  # Compute loss
        
        loss.backward()  # Backward pass
        opt.step()  # Update parameters
        
        running_loss += loss.item() * inputs.size(0)
        monotone_network.clamp_weights()
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

/Users/devarora/ml/submodular_nn/submodular_monotone_net.py:36: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3701.)
  ).T


Epoch 1/100, Loss: 187.0485
Epoch 2/100, Loss: 180.3452
Epoch 3/100, Loss: 173.8180
Epoch 4/100, Loss: 167.4603
Epoch 5/100, Loss: 161.2651
Epoch 6/100, Loss: 155.2264
Epoch 7/100, Loss: 149.3385
Epoch 8/100, Loss: 143.5967
Epoch 9/100, Loss: 137.9971
Epoch 10/100, Loss: 132.5355
Epoch 11/100, Loss: 127.2086
Epoch 12/100, Loss: 122.0126
Epoch 13/100, Loss: 116.9453
Epoch 14/100, Loss: 112.0039
Epoch 15/100, Loss: 107.1862
Epoch 16/100, Loss: 102.4903
Epoch 17/100, Loss: 97.9141
Epoch 18/100, Loss: 93.4569
Epoch 19/100, Loss: 89.1170
Epoch 20/100, Loss: 84.8935
Epoch 21/100, Loss: 80.7856
Epoch 22/100, Loss: 76.7918
Epoch 23/100, Loss: 72.9115
Epoch 24/100, Loss: 69.1439
Epoch 25/100, Loss: 65.4888
Epoch 26/100, Loss: 61.9451
Epoch 27/100, Loss: 58.5120
Epoch 28/100, Loss: 55.1893
Epoch 29/100, Loss: 51.9764
Epoch 30/100, Loss: 48.8725
Epoch 31/100, Loss: 45.8767
Epoch 32/100, Loss: 42.9887
Epoch 33/100, Loss: 40.2081
Epoch 34/100, Loss: 37.5339
Epoch 35/100, Loss: 34.9653
Epoch 36/100,

In [9]:
# input is a binary mask of length 10000 representing whether the i'th element is chosen 
# we precompute the m theta for each vector and then dot it with the mask to get the current thing

In [10]:
# input is a binary mask of length 10000 representing whether the i'th element is chosen 
# we precompute the m theta for each vector and then dot it with the mask to get the current thing